<a href="https://colab.research.google.com/github/skmahin661/AI_guessing_Game/blob/main/AI_Drawing_Guessing_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests

classes_url = "https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt"
classes = requests.get(classes_url).text.splitlines()

os.makedirs("data", exist_ok=True)

for cls in classes:
    cls_name = cls.replace(" ", "_")
    url = f"https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{cls_name}.npy"
    response = requests.get(url)
    with open(f"data/{cls_name}.npy", 'wb') as f:
        f.write(response.content)
    print(f"Downloaded: {cls_name}")


Downloaded: aircraft_carrier
Downloaded: airplane
Downloaded: alarm_clock
Downloaded: ambulance
Downloaded: angel
Downloaded: animal_migration
Downloaded: ant
Downloaded: anvil
Downloaded: apple
Downloaded: arm
Downloaded: asparagus
Downloaded: axe
Downloaded: backpack
Downloaded: banana
Downloaded: bandage
Downloaded: barn
Downloaded: baseball
Downloaded: baseball_bat
Downloaded: basket
Downloaded: basketball
Downloaded: bat
Downloaded: bathtub
Downloaded: beach
Downloaded: bear
Downloaded: beard
Downloaded: bed
Downloaded: bee
Downloaded: belt
Downloaded: bench
Downloaded: bicycle
Downloaded: binoculars
Downloaded: bird
Downloaded: birthday_cake
Downloaded: blackberry
Downloaded: blueberry
Downloaded: book
Downloaded: boomerang
Downloaded: bottlecap
Downloaded: bowtie
Downloaded: bracelet
Downloaded: brain
Downloaded: bread
Downloaded: bridge
Downloaded: broccoli
Downloaded: broom
Downloaded: bucket
Downloaded: bulldozer
Downloaded: bus
Downloaded: bush
Downloaded: butterfly
Download

In [ ]:
import requests

# Download and save the categories.txt file
url = "https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt"
categories_text = requests.get(url).text

with open("categories.txt", "w") as f:
    f.write(categories_text)


In [ ]:
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Load class names
with open("class_names.txt") as f:
    used_class_names = f.read().splitlines()

# Load dataset
X, y = [], []
max_samples = 1000  # increase for better accuracy

for i, name in enumerate(used_class_names):
    path = f"data/{name}.npy"
    if os.path.exists(path):
        images = np.load(path, allow_pickle=True)[:max_samples]
        images = images.reshape(-1, 28, 28, 1) / 255.0  # Normalize
        labels = np.full(images.shape[0], i)
        X.append(images)
        y.append(labels)

X = np.concatenate(X)
y = np.concatenate(y)
X, y = shuffle(X, y, random_state=42)

# One-hot encode labels
y = tf.keras.utils.to_categorical(y, num_classes=len(used_class_names))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)

# Improved CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2),

    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2),

    tf.keras.layers.Conv2D(256, 3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(used_class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

# Train model
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          validation_data=(X_test, y_test),
          epochs=50,
          callbacks=[early_stop])

# Save model
model.save("model.h5")
print("\n✅ Model saved as model.h5")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7350/7350 ━━━━━━━━━━━━━━━━━━━━ 110s 14ms/step - accuracy: 0.1821 - loss: 3.9358 - val_accuracy: 0.4037 - val_loss: 2.5599
Epoch 2/50
7350/7350 ━━━━━━━━━━━━━━━━━━━━ 100s 14ms/step - accuracy: 0.3932 - loss: 2.6187 - val_accuracy: 0.5052 - val_loss: 2.1021
Epoch 3/50
7350/7350 ━━━━━━━━━━━━━━━━━━━━ 100s 14ms/step - accuracy: 0.4361 - loss: 2.4093 - val_accuracy: 0.5193 - val_loss: 2.0108
Epoch 4/50
7350/7350 ━━━━━━━━━━━━━━━━━━━━ 101s 14ms/step - accuracy: 0.4598 - loss: 2.3015 - val_accuracy: 0.5309 - val_loss: 1.9812
Epoch 5/50
7350/7350 ━━━━━━━━━━━━━━━━━━━━ 100s 14ms/step - accuracy: 0.4720 - loss: 2.2400 - val_accuracy: 0.5436 - val_loss: 1.9382
Epoch 6/50
7350/7350 ━━━━━━━━━━━━━━━━━━━━ 139s 13ms/step - accuracy: 0.4836 - loss: 2.1777 - val_accuracy: 0.5570 - val_loss: 1.8837
Epoch 7/50
7350/7350 ━━━━━━━━━━━━━━━━━━━━ 143s 13ms/step - accuracy: 0.4917 - loss: 2.1452 - val_accuracy: 0.5726 - val_loss: 1.8048
Epoch 8/50
7350/7350 ━━━━━━━━━━━━━━━━━━━━ 98s 13ms/step - accuracy: 0.4977 - los


✅ Model saved as model.h5


In [ ]:
with open("class_names.txt", "w") as f:
    for name in used_class_names:
        f.write(name + "\n")
